In [1]:
import sqlite3 as sql

conn = sql.connect("nominations.db")

def send_sql_command(sql_req):
    global conn
    return conn.execute(sql_req).fetchall()
    
query = '''
pragma table_info(nominations)
'''
schema = send_sql_command(query)


query = '''
select * from nominations
limit 10;
'''
first_ten = send_sql_command(query)

for s in schema:
    print(s)
    
for ft in first_ten:
    print(ft)

(0, 'Year', 'INTEGER', 0, None, 0)
(1, 'Category', 'TEXT', 0, None, 0)
(2, 'Nominee', 'TEXT', 0, None, 0)
(3, 'Won', 'INTEGER', 0, None, 0)
(4, 'Movie', 'TEXT', 0, None, 0)
(5, 'Character', 'TEXT', 0, None, 0)
(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul')
(2010, 'Actor -- Supporting 

In [8]:
query = '''
CREATE TABLE ceremonies(
   id integer PRIMARY KEY,
   Year text,
   Host text
);
'''
#conn.execute(query)

insert_query = "INSERT INTO ceremonies (Year, Host) VALUES (?,?);"
years_hosts = [
                (2010, "Steve Martin"),
                (2009, "Hugh Jackman"),
                (2008, "Jon Stewart"),
                (2007, "Ellen DeGeneres"),
                (2006, "Jon Stewart"),
                (2005, "Chris Rock"),
                (2004, "Billy Crystal"),
                (2003, "Steve Martin"),
                (2002, "Whoopi Goldberg"),
                (2001, "Steve Martin"),
                (2000, "Billy Crystal"),
            ]

conn.executemany(insert_query, years_hosts);

query = '''
SELECT * FROM ceremonies
LIMIT 10;
'''
print(send_sql_command(query))


query = '''
PRAGMA table_info(ceremonies);
'''
print(send_sql_command(query))


[(1, '2010', 'Steve Martin'), (2, '2009', 'Hugh Jackman'), (3, '2008', 'Jon Stewart'), (4, '2007', 'Ellen DeGeneres'), (5, '2006', 'Jon Stewart'), (6, '2005', 'Chris Rock'), (7, '2004', 'Billy Crystal'), (8, '2003', 'Steve Martin'), (9, '2002', 'Whoopi Goldberg'), (10, '2001', 'Steve Martin')]
[(0, 'id', 'integer', 0, None, 1), (1, 'Year', 'text', 0, None, 0), (2, 'Host', 'text', 0, None, 0)]


In [9]:
send_sql_command("PRAGMA foreign_keys = ON;")

[]

In [10]:
query = '''
CREATE TABLE nominations_two(
   id integer PRIMARY KEY,
   category text,
   nominee text,
   movie text,
   character text,
   won integer,
   ceremony_id interger,
   FOREIGN KEY(ceremony_id) REFERENCES ceremonies(id)
);
'''
conn.execute(query)

In [11]:
query = '''
SELECT nominations.category, nominations.nominee, nominations.movie, nominations.character, nominations.won, ceremonies.id
FROM nominations
INNER JOIN ceremonies ON
nominations.year == ceremonies.year
;
'''
joined_nominations = send_sql_command(query)

In [12]:
insert_nominations_two = '''
INSERT INTO nominations_two 
(category, nominee, movie, character, won, ceremony_id) VALUES 
(?,?,?,?,?,?);
'''

conn.executemany(insert_nominations_two, joined_nominations)

In [13]:
query = '''
SELECT * FROM nominations_two
LIMIT 5;
'''
print(send_sql_command(query))

[(1, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', 1, 1), (2, 'Actor -- Leading Role', 'James Franco', '127 Hours', 'Aron Ralston', 0, 1), (3, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1), (4, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', 0, 1), (5, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', 0, 1)]


In [14]:
query = '''
DROP TABLE nominations;
'''
print(send_sql_command(query))

[]


In [15]:
query = '''
ALTER TABLE nominations_two
RENAME TO nominations;
'''
print(send_sql_command(query))

[]


In [16]:
create_movies = "create table movies (id integer primary key,movie text);"
create_actors = "create table actors (id integer primary key,actor text);"
create_movies_actors = '''create table movies_actors (id INTEGER PRIMARY KEY,
movie_id INTEGER references movies(id), actor_id INTEGER references actors(id));
'''
conn.execute(create_movies)
conn.execute(create_actors)
conn.execute(create_movies_actors)

In [17]:
insert_movies = "insert into movies (movie) select distinct movie from nominations;"
insert_actors = "insert into actors (actor) select distinct nominee from nominations;"
conn.execute(insert_movies)
conn.execute(insert_actors)

print(conn.execute("select * from movies limit 5;").fetchall())
print(conn.execute("select * from actors limit 5;").fetchall())

[(1, "The King's Speech"), (2, '127 Hours'), (3, 'Biutiful'), (4, 'True Grit'), (5, 'The Social Network')]
[(1, 'Colin Firth'), (2, 'James Franco'), (3, 'Javier Bardem'), (4, 'Jeff Bridges'), (5, 'Jesse Eisenberg')]


In [19]:
pairs_query = "select movie,nominee from nominations;"
movie_actor_pairs = conn.execute(pairs_query).fetchall()

join_table_insert = "insert into movies_actors (movie_id, actor_id) values ((select id from movies where movie == ?),(select id from actors where actor == ?));"
conn.executemany(join_table_insert,movie_actor_pairs)

print(conn.execute("select * from movies_actors limit 5;").fetchall())

[(1, 1, 1), (2, 2, 2), (3, 3, 3), (4, 4, 4), (5, 5, 5)]
